In [ ]:
import streamlit as st
import requests
import pandas as pd
import plotly.graph_objs as go

stock='APPL'


def get_intraday_data_alpha(symbol):
    API_KEY = st.secrets["api"]["alpha_vantage_api"]

    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=5min&apikey={API_KEY}"


    try:
        response = requests.get(url)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

    if response.status_code != 200:
        st.error(f"Error fetching data for {symbol}")
        return None
    
    data = response.json()
    time_series_key = "Time Series (5min)"
    
    if time_series_key not in data:
        st.error(f"No data found for {symbol}. Check the ticker or API limits.")
        return None

    # Convert JSON data to DataFrame
    df = pd.DataFrame.from_dict(data[time_series_key], orient="index")
    
    # Rename columns
    df = df.rename(columns={
        "1. open": "Open",
        "2. high": "High",
        "3. low": "Low",
        "4. close": "Close",
        "5. volume": "Volume"
    })
    
    df.index = pd.to_datetime(df.index)  # Convert index to datetime
    df = df.sort_index()  # Ensure chronological order
    
    return df

stock='AAPL'
df_alpha = get_intraday_data_alpha(stock)


In [21]:
df_alpha

,Open,High,Low,Close,Volume
2025-04-16 11:40:00,198.0200,198.0285,197.5700,197.6900,302056
2025-04-16 11:45:00,197.6900,197.7435,197.0000,197.3800,349071
2025-04-16 11:50:00,197.3700,197.4500,197.0000,197.1700,391177
2025-04-16 11:55:00,197.1897,197.1900,196.7100,196.7500,419119
2025-04-16 12:00:00,196.7550,197.0000,196.5000,196.9800,450929
...,...,...,...,...,...
2025-04-16 19:35:00,195.1800,195.1900,194.4100,194.5600,4556
2025-04-16 19:40:00,194.5600,194.9000,194.5585,194.9000,2202
2025-04-16 19:45:00,194.7600,195.1000,194.7100,195.0000,2632
2025-04-16 19:50:00,194.9500,195.1000,194.6600,194.7900,6414


In [25]:
import datetime
from datetime import datetime, timedelta
stock='APPL'

def get_daily_data_polygon(symbol):
        # 🔹 Your Polygon.io API Key
        API_KEY = st.secrets['api']['polygon_api']  # Replace with your actual API key

        # 🔹 Define Date Range (Last 5 Years)
        end_date = datetime.today().strftime('%Y-%m-%d')  # Today's date
        start_date = (datetime.today() - timedelta(days=5*365)).strftime('%Y-%m-%d')  # 5 years ago
        url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{start_date}/{end_date}?adjusted=true&sort=asc&limit=5000&apiKey={API_KEY}"

        # 🔹 Fetch Data
        response = requests.get(url)
        data = response.json()

        # 🔹 Process Data
        if "results" in data:
            df = pd.DataFrame(data["results"])
            
            # Convert timestamp to readable date
            df["date"] = pd.to_datetime(df["t"], unit="ms")
            
            # Select relevant columns
            df = df[["date", "o", "h", "l", "c", "v"]]
            
            # Rename columns
            df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
            df.set_index('Date', inplace=True)
            return df
df_polygon=get_daily_data_polygon('AAPL')

In [26]:
def plot_candlestick(stock_data, symbol, time_range):
    end_date = datetime.today()
    time_range_options = ["5d","1m", "3m", "6m", "1y", "2y", "5y", "YTD", "max"]
    if time_range =='5d':
        start_date = end_date - timedelta(days=5)
    elif time_range == '1m':
        start_date = end_date - timedelta(days=30)
    elif time_range == '3m':
        start_date = end_date - timedelta(days=90)
    elif time_range == '6m':
        start_date = end_date - timedelta(days=180)
    elif time_range == '1y':
        start_date = end_date - timedelta(days=365)
    elif time_range == '2y':
        start_date = end_date - timedelta(days=2*365)
    elif time_range == '5y':
        start_date = end_date - timedelta(days=5*365)  # Approximate 5 years

    elif time_range == 'YTD':
        start_date = datetime(end_date.year, 1, 1)

    stock_data = stock_data.loc[start_date:end_date]
    candlestick_chart = go.Figure(data=[
        go.Candlestick(
            x=stock_data.index,
            open=stock_data['Open'],
            high=stock_data['High'],
            low=stock_data['Low'],
            close=stock_data['Close']
        )
    ])

    candlestick_chart.update_layout(
        title=f"{symbol} Candlestick Chart ({time_range})",
        xaxis_rangeslider_visible=False
    
    )
    return candlestick_chart

In [31]:
plot_candlesticak(df_polygon, stock, '2y')

NameError: name 'plot_candlesticak' is not defined

In [30]:
%pip install nbformat>=4.2.0

Note: you may need to restart the kernel to use updated packages.
